In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset (y, psi, omega) assuming 10000 rows
num_samples = 10000
n, d = 150, 512  # Dimensions given in problem

y = torch.randn(num_samples, n, 1)
psi = torch.randn(num_samples, n, d)
omega = torch.randn(num_samples, n, 1)

# Compute Least Squares Estimate h_LS using Moore-Penrose pseudo-inverse
h_LS = torch.linalg.pinv(psi) @ (y - omega)  # Shape: (num_samples, d, 1)
h_LS = h_LS.squeeze(-1)  # Shape: (num_samples, d)

# Prepare dataset
train_dataset = TensorDataset(h_LS, h_LS)  # Input and output are both h_LS
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define candidate operations for the search space
OPS = {
    'conv_3x3': lambda C: nn.Conv2d(C, C, kernel_size=3, stride=1, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv2d(C, C, kernel_size=5, stride=1, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv2d(C, C, 1, stride=1, bias=False), nn.BatchNorm2d(C)),
    'zero': lambda C: nn.ZeroPad2d(0),
}

# Mixed operation layer with probability-based selection
class MixedOp(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops)))  # Learnable probabilities
    
    def forward(self, x):
        weights = F.softmax(self.alphas, dim=0)  # Normalize with softmax
        return sum(w * op(x) for w, op in zip(weights, self.ops))

# Searchable Neural Network with architecture selection
class SearchNetwork(nn.Module):
    def __init__(self, C, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList([MixedOp(C) for _ in range(num_layers)])
        self.output_layer = nn.Conv2d(C, C, kernel_size=1, stride=1, bias=False)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.output_layer(x)

# Training function with truncated reverse-mode AD for NAS
def train(model, train_loader, arch_optimizer, model_optimizer, criterion, unroll_steps=1):
    model.train()
    for x, y in train_loader:
        x, y = x.unsqueeze(1), y.unsqueeze(1)  # Add channel dimension for CNN layers
        x, y = x.to(device), y.to(device)
        
        # Compute loss and update model parameters
        model_optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, y)
        loss.backward()
        model_optimizer.step()
        
        # Architecture update using truncated differentiation
        arch_optimizer.zero_grad()
        with torch.no_grad():
            temp_model = SearchNetwork(d).to(device)
            temp_model.load_state_dict(model.state_dict())
        for _ in range(unroll_steps):
            temp_output = temp_model(x)
            temp_loss = criterion(temp_output, y)
            temp_loss.backward()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_layers = 3
num_epochs = 20

# Initialize model and optimizers
model = SearchNetwork(d).to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model with NAS
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

# Extract final architecture based on highest probability operations
def extract_final_architecture(model):
    final_ops = []
    for layer in model.layers:
        best_op_idx = torch.argmax(layer.alphas).item()
        best_op_name = list(OPS.keys())[best_op_idx]
        final_ops.append(best_op_name)
    return final_ops

final_architecture = extract_final_architecture(model)
print("Final Architecture:", final_architecture)

# Save the best model
torch.save(model.state_dict(), "best_model.pth")

print("Neural architecture search completed.")

with real and imaginary separate

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions given in problem

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS using Moore-Penrose pseudo-inverse
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)  # Shape: (num_samples, d, 1)
h_LS_complex = h_LS_complex.squeeze(-1)  # Shape: (num_samples, d)

# Split complex numbers into real and imaginary parts
h_LS_real = h_LS_complex.real  # Shape: (num_samples, d)
h_LS_imag = h_LS_complex.imag  # Shape: (num_samples, d)

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)  # Inputs and outputs for training
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Define candidate operations for the search space
OPS = {
    'conv_3x3': lambda C: nn.Conv2d(C, C, kernel_size=3, stride=1, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv2d(C, C, kernel_size=5, stride=1, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv2d(C, C, 1, stride=1, bias=False), nn.BatchNorm2d(C)),
    'zero': lambda C: nn.ZeroPad2d(0),
}

# Mixed operation layer with probability-based selection
class MixedOp(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops)))  # Learnable probabilities
    
    def forward(self, x):
        weights = F.softmax(self.alphas, dim=0)  # Normalize with softmax
        return sum(w * op(x) for w, op in zip(weights, self.ops))

# Searchable Neural Network with architecture selection
class SearchNetwork(nn.Module):
    def __init__(self, C, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList([MixedOp(C) for _ in range(num_layers)])
        self.output_layer = nn.Conv2d(C, C, kernel_size=1, stride=1, bias=False)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.output_layer(x)

# Training function with truncated reverse-mode AD for NAS
def train(model, train_loader, arch_optimizer, model_optimizer, criterion, unroll_steps=1):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)  # Add channel dimension
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_in.to(device), real_out.to(device), imag_out.to(device)
        
        # Compute loss and update model parameters
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        # Architecture update using truncated differentiation
        arch_optimizer.zero_grad()
        with torch.no_grad():
            temp_model = SearchNetwork(d).to(device)
            temp_model.load_state_dict(model.state_dict())
        for _ in range(unroll_steps):
            real_temp_pred, imag_temp_pred = temp_model(real_in), temp_model(imag_in)
            temp_loss = criterion(real_temp_pred, real_out) + criterion(imag_temp_pred, imag_out)
            temp_loss.backward()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_layers = 3
num_epochs = 20

# Initialize model and optimizers
model = SearchNetwork(d).to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model with NAS
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

# Extract final architecture based on highest probability operations
def extract_final_architecture(model):
    final_ops = []
    for layer in model.layers:
        best_op_idx = torch.argmax(layer.alphas).item()
        best_op_name = list(OPS.keys())[best_op_idx]
        final_ops.append(best_op_name)
    return final_ops

final_architecture = extract_final_architecture(model)
print("Final Architecture:", final_architecture)

# Save the best model
torch.save(model.state_dict(), "best_model.pth")
print("Neural architecture search completed.")

with the initial 2 convolutional layers that extract features from the input and the decoder module at the end

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)  # Shape: (num_samples, d, 1)
h_LS_complex = h_LS_complex.squeeze(-1)  # Shape: (num_samples, d)

# Split into real and imaginary components
h_LS_real = h_LS_complex.real  # Shape: (num_samples, d)
h_LS_imag = h_LS_complex.imag  # Shape: (num_samples, d)

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Feature Extraction Layers
class FeatureExtractor(nn.Module):
    def __init__(self, input_type):
        super().__init__()
        if input_type == 'vector':
            self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
            self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        else:
            self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
            self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# NAS Candidate Operations
OPS = {
    'conv_3x3': lambda C: nn.Conv2d(C, C, kernel_size=3, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv2d(C, C, kernel_size=5, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv2d(C, C, 1, bias=False), nn.BatchNorm2d(C)),
    'zero': lambda C: nn.ZeroPad2d(0),
}

# Mixed Operation Layer
class MixedOp(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops)))

    def forward(self, x):
        weights = F.softmax(self.alphas, dim=0)
        return sum(w * op(x) for w, op in zip(weights, self.ops))

# Searchable Neural Network
class SearchNetwork(nn.Module):
    def __init__(self, C, num_layers=3):
        super().__init__()
        self.layers = nn.ModuleList([MixedOp(C) for _ in range(num_layers)])
        self.output_layer = nn.Conv2d(C, C, kernel_size=1, bias=False)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return self.output_layer(x)

# Decoder Module
class Decoder(nn.Module):
    def __init__(self, input_type):
        super().__init__()
        self.fc = nn.Linear(512, 512)
        
        if input_type == 'vector':
            self.sep_conv1 = nn.Conv1d(32, 16, kernel_size=3, padding=1, groups=16)
            self.sep_conv2 = nn.Conv1d(16, 1, kernel_size=3, padding=1, groups=1)
            self.final_conv = nn.Conv1d(1, 1, kernel_size=3, padding=1)
        else:
            self.sep_conv1 = nn.Conv2d(32, 16, kernel_size=3, padding=1, groups=16)
            self.sep_conv2 = nn.Conv2d(16, 1, kernel_size=3, padding=1, groups=1)
            self.final_conv = nn.Conv2d(1, 1, kernel_size=3, padding=1)
        
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = F.relu(self.sep_conv1(x))
        x = F.relu(self.sep_conv2(x))
        return self.final_conv(x)

# Complete Model
class FullModel(nn.Module):
    def __init__(self, input_type):
        super().__init__()
        self.feature_extractor = FeatureExtractor(input_type)
        self.search_network = SearchNetwork(32)
        self.decoder = Decoder(input_type)
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.search_network(x)
        return self.decoder(x)

# Training Function
def train(model, train_loader, arch_optimizer, model_optimizer, criterion, unroll_steps=1):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)  # Add channel dimension
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_in.to(device), real_out.to(device), imag_out.to(device)
        
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        arch_optimizer.zero_grad()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20

# Initialize model
model = FullModel('vector').to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

torch.save(model.state_dict(), "best_model.pth")
print("Training completed.")

with the DAG denoise cells with 4 nodes in each cell and the sequence of denoise cells have 10 cells.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)
h_LS_complex = h_LS_complex.squeeze(-1)

# Split into real and imaginary components
h_LS_real = h_LS_complex.real
h_LS_imag = h_LS_complex.imag

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Feature Extraction Layers
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# NAS Candidate Operations
OPS = {
    'conv_3x3': lambda C: nn.Conv1d(C, C, kernel_size=3, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv1d(C, C, kernel_size=5, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv1d(C, C, 1, bias=False), nn.BatchNorm1d(C)),
    'zero': lambda C: nn.ZeroPad1d(0),
}

# Denoise Cell
class DenoiseCell(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops), 3))  # 3 edges per cell

    def forward(self, inputs):
        assert len(inputs) == 2, "Each denoise cell must take two inputs."
        node_outputs = [inputs[0], inputs[1], torch.zeros_like(inputs[0]), torch.zeros_like(inputs[0])]
        
        for i in range(3):  # Three edges per cell
            weights = F.softmax(self.alphas[:, i], dim=0)
            node_outputs[i + 1] = sum(w * op(node_outputs[i]) for w, op in zip(weights, self.ops))
        
        return node_outputs[-1]

# Sequence of 10 Denoise Cells
class DenoiseModule(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.cells = nn.ModuleList([DenoiseCell(C) for _ in range(10)])

    def forward(self, x):
        out1, out2 = x, x  # First two cells take same input
        outputs = [out1, out2]
        
        for i in range(10):
            out = self.cells[i]([outputs[-2], outputs[-1]])
            outputs.append(out)
        
        return outputs[-1]

# Decoder Module
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(512, 512)
        self.sep_conv1 = nn.Conv1d(32, 16, kernel_size=3, padding=1, groups=16)
        self.sep_conv2 = nn.Conv1d(16, 1, kernel_size=3, padding=1, groups=1)
        self.final_conv = nn.Conv1d(1, 1, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = F.relu(self.sep_conv1(x))
        x = F.relu(self.sep_conv2(x))
        return self.final_conv(x)

# Complete Model
class FullModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.denoise_module = DenoiseModule(32)
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.denoise_module(x)
        return self.decoder(x)

# Training Function
def train(model, train_loader, arch_optimizer, model_optimizer, criterion):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)  # Add channel dimension
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_in.to(device), real_out.to(device), imag_out.to(device)
        
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        arch_optimizer.zero_grad()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20

# Initialize model
model = FullModel().to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

torch.save(model.state_dict(), "best_model.pth")
print("Training completed.")

with 6 edges in each denoise cell

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)
h_LS_complex = h_LS_complex.squeeze(-1)

# Split into real and imaginary components
h_LS_real = h_LS_complex.real
h_LS_imag = h_LS_complex.imag

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Feature Extraction Layers
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# NAS Candidate Operations
OPS = {
    'conv_3x3': lambda C: nn.Conv1d(C, C, kernel_size=3, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv1d(C, C, kernel_size=5, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv1d(C, C, 1, bias=False), nn.BatchNorm1d(C)),
    'zero': lambda C: nn.ZeroPad1d(0),
}

# Denoise Cell with DAG Structure
class DenoiseCell(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops), 6))  # 6 edges in DAG

    def forward(self, inputs):
        assert len(inputs) == 2, "Each denoise cell must take two inputs."
        node_outputs = [inputs[0], inputs[1], torch.zeros_like(inputs[0]), torch.zeros_like(inputs[0])]
        
        edges = [(0, 2), (0, 3), (0, 4), (2, 3), (2, 4), (3, 4)]
        for edge_idx, (src, dest) in enumerate(edges):
            weights = F.softmax(self.alphas[:, edge_idx], dim=0)
            node_outputs[dest] += sum(w * op(node_outputs[src]) for w, op in zip(weights, self.ops))
        
        return node_outputs[4]  # Output of last node

# Sequence of 10 Denoise Cells
class DenoiseModule(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.cells = nn.ModuleList([DenoiseCell(C) for _ in range(10)])

    def forward(self, x):
        out1, out2 = x, x
        outputs = [out1, out2]
        
        for i in range(10):
            out = self.cells[i]([outputs[-2], outputs[-1]])
            outputs.append(out)
        
        return outputs[-1]

# Decoder Module
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(512, 512)
        self.sep_conv1 = nn.Conv1d(32, 16, kernel_size=3, padding=1, groups=16)
        self.sep_conv2 = nn.Conv1d(16, 1, kernel_size=3, padding=1, groups=1)
        self.final_conv = nn.Conv1d(1, 1, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = F.relu(self.sep_conv1(x))
        x = F.relu(self.sep_conv2(x))
        return self.final_conv(x)

# Complete Model
class FullModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.denoise_module = DenoiseModule(32)
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.denoise_module(x)
        return self.decoder(x)

# Training Function
def train(model, train_loader, arch_optimizer, model_optimizer, criterion):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_in.to(device), real_out.to(device), imag_out.to(device)
        
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        arch_optimizer.zero_grad()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20

# Initialize model
model = FullModel().to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

torch.save(model.state_dict(), "best_model.pth")
print("Training completed.")

with base 2 denoise cells sorted

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)
h_LS_complex = h_LS_complex.squeeze(-1)

# Split into real and imaginary components
h_LS_real = h_LS_complex.real
h_LS_imag = h_LS_complex.imag

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Feature Extraction Layers
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# NAS Candidate Operations
OPS = {
    'conv_3x3': lambda C: nn.Conv1d(C, C, kernel_size=3, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv1d(C, C, kernel_size=5, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv1d(C, C, 1, bias=False), nn.BatchNorm1d(C)),
    'zero': lambda C: nn.ZeroPad1d(0),
}

# Denoise Cell with DAG Structure
class DenoiseCell(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops), 6))  # 6 edges in DAG

    def forward(self, inputs):
        assert len(inputs) == 2, "Each denoise cell must take two inputs."
        node_outputs = [inputs[0], inputs[1], torch.zeros_like(inputs[0]), torch.zeros_like(inputs[0])]
        
        edges = [(0, 2), (0, 3), (0, 4), (2, 3), (2, 4), (3, 4)]
        for edge_idx, (src, dest) in enumerate(edges):
            weights = F.softmax(self.alphas[:, edge_idx], dim=0)
            node_outputs[dest] += sum(w * op(node_outputs[src]) for w, op in zip(weights, self.ops))
        
        return node_outputs[4]  # Output of last node

# Sequence of 10 Denoise Cells
class DenoiseModule(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.cells = nn.ModuleList([DenoiseCell(C) for _ in range(10)])

    def forward(self, x):
        out1, out2 = x, x  # Initial input duplicated for first denoise cell
        outputs = [out1, out2]
        
        for i in range(10):
            if i == 0:
                # First cell: use feature extraction output twice
                out = self.cells[i]([out1, out1])
            elif i == 1:
                # Second cell: use output of first cell and original feature extraction output
                out = self.cells[i]([outputs[1], out1])
            else:
                # Remaining cells: use last 2 outputs
                out = self.cells[i]([outputs[-2], outputs[-1]])
            
            outputs.append(out)
        
        return outputs[-1]

# Decoder Module
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(512, 512)
        self.sep_conv1 = nn.Conv1d(32, 16, kernel_size=3, padding=1, groups=16)
        self.sep_conv2 = nn.Conv1d(16, 1, kernel_size=3, padding=1, groups=1)
        self.final_conv = nn.Conv1d(1, 1, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = F.relu(self.sep_conv1(x))
        x = F.relu(self.sep_conv2(x))
        return self.final_conv(x)

# Complete Model
class FullModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.denoise_module = DenoiseModule(32)
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.denoise_module(x)
        return self.decoder(x)

# Training Function
def train(model, train_loader, arch_optimizer, model_optimizer, criterion):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_in.to(device), real_out.to(device), imag_out.to(device)
        
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        arch_optimizer.zero_grad()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20

# Initialize model
model = FullModel().to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

torch.save(model.state_dict(), "best_model.pth")
print("Training completed.")

with aggrehgation done in cases where there at mutiple inputs in the nodes

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)
h_LS_complex = h_LS_complex.squeeze(-1)

# Split into real and imaginary components
h_LS_real = h_LS_complex.real
h_LS_imag = h_LS_complex.imag

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Feature Extraction Layers
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# NAS Candidate Operations
OPS = {
    'conv_3x3': lambda C: nn.Conv1d(C, C, kernel_size=3, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv1d(C, C, kernel_size=5, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv1d(C, C, 1, bias=False), nn.BatchNorm1d(C)),
    'zero': lambda C: nn.ZeroPad1d(0),
}

# Denoise Cell with DAG Structure
class DenoiseCell(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.C = C
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops), 6))  # 6 edges in DAG

        # 1x1 convolution to reduce dimensions after concatenation for node 3
        self.conv1x1_node3 = nn.Conv1d(3 * C, C, kernel_size=1, bias=False)

    def forward(self, inputs):
        assert len(inputs) == 2, "Each denoise cell must take two inputs."
        # Concatenation for nodes 0 and 2
        node0 = torch.cat(inputs, dim=1)
        node2 = torch.cat(inputs, dim=1)
        
        node_outputs = [
            F.relu(node0),  # Node 0: concatenation of 2 inputs
            F.relu(node2),  # Node 1: concatenation of 2 inputs
            torch.zeros_like(inputs[0]),  # Node 2
            torch.zeros_like(inputs[0]),  # Node 3
            torch.zeros_like(inputs[0]),  # Node 4
        ]
        
        edges = [(0, 2), (0, 3), (0, 4), (2, 3), (2, 4), (3, 4)]
        for edge_idx, (src, dest) in enumerate(edges):
            weights = F.softmax(self.alphas[:, edge_idx], dim=0)
            aggregated_output = sum(w * op(node_outputs[src]) for w, op in zip(weights, self.ops))

            if dest == 3:
                # Node 3: Concatenate 3 inputs and reduce dimensions with 1x1 convolution
                node_outputs[dest] = F.relu(self.conv1x1_node3(torch.cat([node_outputs[0], node_outputs[1], aggregated_output], dim=1)))
            elif dest == 2:
                node_outputs[dest] += aggregated_output
            else:
                node_outputs[dest] += aggregated_output
        
        return node_outputs[4]  # Output of last node

# Sequence of 10 Denoise Cells
class DenoiseModule(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.cells = nn.ModuleList([DenoiseCell(C) for _ in range(10)])

    def forward(self, x):
        out1, out2 = x, x  # Initial input duplicated for first denoise cell
        outputs = [out1, out2]
        
        for i in range(10):
            if i == 0:
                # First cell: use feature extraction output twice
                out = self.cells[i]([out1, out1])
            elif i == 1:
                # Second cell: use output of first cell and original feature extraction output
                out = self.cells[i]([outputs[1], out1])
            else:
                # Remaining cells: use last 2 outputs
                out = self.cells[i]([outputs[-2], outputs[-1]])
            
            outputs.append(out)
        
        return outputs[-1]

# Decoder Module
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(512, 512)
        self.sep_conv1 = nn.Conv1d(32, 16, kernel_size=3, padding=1, groups=16)
        self.sep_conv2 = nn.Conv1d(16, 1, kernel_size=3, padding=1, groups=1)
        self.final_conv = nn.Conv1d(1, 1, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = F.relu(self.sep_conv1(x))
        x = F.relu(self.sep_conv2(x))
        return self.final_conv(x)

# Complete Model
class FullModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.denoise_module = DenoiseModule(32)
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.denoise_module(x)
        return self.decoder(x)

# Training Function
def train(model, train_loader, arch_optimizer, model_optimizer, criterion):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_out.to(device), real_out.to(device), imag_out.to(device)
        
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        arch_optimizer.zero_grad()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20

# Initialize model
model = FullModel().to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

torch.save(model.state_dict(), "best_model.pth")
print("Training completed.")

TRIAL

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Generate synthetic dataset with complex values
num_samples = 10000
n, d = 150, 512  # Dimensions

y_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)
psi_complex = torch.randn(num_samples, n, d, dtype=torch.cfloat)
omega_complex = torch.randn(num_samples, n, 1, dtype=torch.cfloat)

# Compute Least Squares Estimate h_LS
h_LS_complex = torch.linalg.pinv(psi_complex) @ (y_complex - omega_complex)
h_LS_complex = h_LS_complex.squeeze(-1)

# Split into real and imaginary components
h_LS_real = h_LS_complex.real
h_LS_imag = h_LS_complex.imag

# Prepare dataset
train_dataset = TensorDataset(h_LS_real, h_LS_imag, h_LS_real, h_LS_imag)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Feature Extraction Layers
class FeatureExtractor(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        return x

# NAS Candidate Operations
OPS = {
    'conv_3x3': lambda C: nn.Conv1d(C, C, kernel_size=3, padding=1, bias=False),
    'conv_5x5': lambda C: nn.Conv1d(C, C, kernel_size=5, padding=2, bias=False),
    'identity': lambda C: nn.Identity(),
    'skip_connection': lambda C: nn.Sequential(nn.Conv1d(C, C, 1, bias=False), nn.BatchNorm1d(C)),
    'zero': lambda C: nn.ZeroPad1d(0),
}

# Denoise Cell with DAG Structure
class DenoiseCell(nn.Module):
    def _init_(self, C):
        super()._init_()
        self.C = C
        self.ops = nn.ModuleList([op(C) for op in OPS.values()])
        self.alphas = nn.Parameter(torch.randn(len(self.ops), 6))  # 6 edges in DAG

        # 1x1 convolution to reduce dimensions after concatenation for node 3
        self.conv1x1_node3 = nn.Conv1d(3 * C, C, kernel_size=1, bias=False)

    def pad_and_concat(self, inputs):
        """Finds the max size, zero-pads smaller inputs, and concatenates along the channel dimension."""
        max_size = max(inp.shape[2] for inp in inputs)  # Find the largest temporal dimension
        padded_inputs = [
            F.pad(inp, (0, max_size - inp.shape[2])) if inp.shape[2] < max_size else inp
            for inp in inputs
        ]
        return torch.cat(padded_inputs, dim=1)

    def forward(self, inputs):
        assert len(inputs) == 2, "Each denoise cell must take two inputs."
        
        node_outputs = [None] * 4  # Initialize all nodes
        
        # Node 0: Receives both inputs, pads them, and concatenates
        node_outputs[0] = F.relu(self.pad_and_concat(inputs))
        
        # Node 1: Receives only node 0's output
        node_outputs[1] = F.relu(node_outputs[0])
        
        # Node 2: Receives input from node 0 and node 1 (must pad and concat)
        node_outputs[2] = F.relu(self.pad_and_concat([node_outputs[0], node_outputs[1]]))
        
        # Node 3: Receives inputs from nodes 0, 1, and 2 (pad and concat)
        padded_inputs_node3 = self.pad_and_concat([node_outputs[0], node_outputs[1], node_outputs[2]])
        node_outputs[3] = F.relu(self.conv1x1_node3(padded_inputs_node3))
        
        return node_outputs[3]  # Output of node 3

# Sequence of 10 Denoise Cells
class DenoiseModule(nn.Module):
    def __init__(self, C):
        super().__init__()
        self.cells = nn.ModuleList([DenoiseCell(C) for _ in range(10)])

    def forward(self, x):
        out1, out2 = x, x  # Initial input duplicated for first denoise cell
        outputs = [out1, out2]
        
        for i in range(10):
            if i == 0:
                # First cell: use feature extraction output twice
                out = self.cells[i]([out1, out1])
            elif i == 1:
                # Second cell: use output of first cell and original feature extraction output
                out = self.cells[i]([outputs[1], out1])
            else:
                # Remaining cells: use last 2 outputs
                out = self.cells[i]([outputs[-2], outputs[-1]])
            
            outputs.append(out)
        
        return outputs[-1]

# Decoder Module
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(512, 512)
        self.sep_conv1 = nn.Conv1d(32, 16, kernel_size=3, padding=1, groups=16)
        self.sep_conv2 = nn.Conv1d(16, 1, kernel_size=3, padding=1, groups=1)
        self.final_conv = nn.Conv1d(1, 1, kernel_size=3, padding=1)
    
    def forward(self, x):
        x = F.relu(self.fc(x))
        x = F.relu(self.sep_conv1(x))
        x = F.relu(self.sep_conv2(x))
        return self.final_conv(x)

# Complete Model
class FullModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.feature_extractor = FeatureExtractor()
        self.denoise_module = DenoiseModule(32)
        self.decoder = Decoder()
    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.denoise_module(x)
        return self.decoder(x)

# Training Function
def train(model, train_loader, arch_optimizer, model_optimizer, criterion):
    model.train()
    for real_in, imag_in, real_out, imag_out in train_loader:
        real_in, imag_in = real_in.unsqueeze(1), imag_in.unsqueeze(1)
        real_out, imag_out = real_out.unsqueeze(1), imag_out.unsqueeze(1)
        real_in, imag_in, real_out, imag_out = real_in.to(device), imag_out.to(device), real_out.to(device), imag_out.to(device)
        
        model_optimizer.zero_grad()
        real_pred, imag_pred = model(real_in), model(imag_in)
        loss = criterion(real_pred, real_out) + criterion(imag_pred, imag_out)
        loss.backward()
        model_optimizer.step()
        
        arch_optimizer.zero_grad()
        arch_optimizer.step()

# Hyperparameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 20

# Initialize model
model = FullModel().to(device)
arch_optimizer = optim.Adam(model.parameters(), lr=0.003)
model_optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train model
for epoch in range(num_epochs):
    train(model, train_loader, arch_optimizer, model_optimizer, criterion)
    print(f"Epoch {epoch+1}/{num_epochs} completed.")

torch.save(model.state_dict(), "best_model.pth")
print("Training completed.")